In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-02-25 02:27:33--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-02-25 02:27:34 (6.63 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase" ])
vine_df.show(20)


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3LI5TRP3YIDQL|          5|            0|          0|   N|                Y|
|R3LGC3EKEG84PX|          5|            0|          0|   N|                Y|
| R9PYL3OYH55QY|          5|            0|          1|   N|                Y|
|R3PWBAWUS4NT0Q|          3|            0|          0|   N|                Y|
|R15LYP3O51UU9E|          5|            0|          0|   N|                Y|
|R1AD7L0CC3DSRI|          5|            0|          0|   N|                Y|
|R32FE8Y45QV434|          5|            0|          0|   N|                Y|
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|                Y|
|R3H4FXX6Q7I37D|          4|            0|          0|   N|                Y|
|R30L5PET7LFFDC|          5|            1|          1|   N|     

In [ ]:
import numpy as np
non_zero_data = vine_df.filter("total_votes > 0")

In [ ]:
# Filter vine datatable and retreive all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%

filtered_df = vine_df.filter(non_zero_data["helpful_votes"]/non_zero_data["total_votes"] >= 0.5).show(20)


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|                Y|
|R30L5PET7LFFDC|          5|            1|          1|   N|                Y|
|  REFRE1LEKLAF|          4|            1|          1|   N|                Y|
|R25Z4ADMYBHR66|          4|            1|          2|   N|                Y|
|R2SGL2ZAPGJH9I|          5|            1|          1|   N|                N|
| RZR1QCHTL8Q4Z|          5|            1|          1|   N|                N|
|R2M8ZRIWBPQGRR|          4|            1|          1|   N|                Y|
|R3C2J53J8UDDPW|          5|            5|          5|   N|                N|
|R37RFLBIGKHA9L|          4|            1|          1|   N|                N|
|R39NGBRCQYAJRH|          4|            4|          4|   N|     

In [ ]:
# Create new df that retreives all the rows where a review was written as part of the vine program paid ( vine == 'Y')
Y_df = filtered_df.filter(filtered_df["vine"] == "y").show()

In [ ]:

# Create new df that retreives all the rows where a review was written as part of the vine program unpaid ( vine == 'N')
N_df = filtered_df.filter(filtered_df["vine"] == "N").show()

In [ ]:
# Determine the total number of reviews, the number of 5 star reviews for the two types of reviews\
five_star_df = filtered_df.filter(filtered_df["star_rating"] == 5)

In [ ]:
# Determine the number of 5 star reviews by using the .count on the 5 star df
five_star_number = five_star_df.count()

In [ ]:
# Determine the total reviews by counting the total rows in the filtered_df
filtered_number = filtered_df.count()

In [ ]:
# Use the count of 5 star reviews and divide by the total reviews 
percent_of_five_star = (five_star_number / filtered_number) * 100

In [ ]:
# Determine the percentage of five star reviews for each type of review
verified_five_star = five_star_df.filter(five_star_df["verified_purchase"] == "Y").count()

(verified_five_star / filtered_df.filter(filtered_df["verified_purchase"] == "Y").count()) * 100

In [ ]:
# Determine the percentage of five star reviews for each type of review
non_verified_five_star = five_star_df.filter(five_star_df["verified_purchase"] == "N").count()

(non_verified_five_star / filtered_df.filter(filtered_df["verified_purchase"] == "N").count()) * 100